In [1]:
import pytorch_lightning as pl

import torch
import toml

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

from aste.train.models.model import ASTEModel
from aste.train.data_providers import DataModule

/home/khaymonenko/python3.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inference_recipe = toml.load("/home/khaymonenko/aste/aste/train/recipes/example_recipes/inference_recipes/mvp-inference-recipe.toml")
train_recipe = toml.load("/home/khaymonenko/aste/aste/train/recipes/example_recipes/experiment-04-t5-1.7B-freeze-300-sentences-mvp.toml")

In [3]:
inference_recipe

{'gpu': 6,
 'checkpoint': '/home/khaymonenko/aste/aste/train/checkpoints/blacklist_pos/epoch=9-step=14099.ckpt',
 'result_path': '/home/khaymonenko/aste/aste/train/inference-result-beamsearch.txt',
 'model_runner_name': 'GenerativeMVPModelRunner',
 'n_orders': 6,
 'generation': {'num_beams': 3, 'temperature': 0.3},
 'dataloader': {'dataset': {'data_path': '/home/khaymonenko/aste/data/bank_3200/test_full.txt',
   'datareader_class_name': 'SentenceBankDataReader',
   'dataset_class_name': 'MVPDataset',
   'input_max_length': 512,
   'output_max_length': 128,
   'nlp': 'ru_core_news_lg'},
  'dataloader': {'batch_size': 16, 'num_workers': 8}}}

In [4]:
train_recipe

{'dataloaders': {'train': {'dataset': {'data_path': '/home/khaymonenko/aste/data/bank_3200/train_full.txt',
    'datareader_class_name': 'SentenceBankDataReader',
    'dataset_class_name': 'MVPDataset',
    'input_max_length': 512,
    'output_max_length': 128,
    'nlp': 'ru_core_news_lg'},
   'dataloader': {'batch_size': 8, 'num_workers': 8}},
  'dev': {'dataset': {'data_path': '/home/khaymonenko/aste/data/bank_3200/dev_full.txt',
    'datareader_class_name': 'SentenceBankDataReader',
    'dataset_class_name': 'MVPDataset',
    'input_max_length': 512,
    'output_max_length': 128,
    'nlp': 'ru_core_news_lg'},
   'dataloader': {'batch_size': 16, 'num_workers': 8}},
  'test': {'dataset': {'data_path': '/home/khaymonenko/aste/data/bank_3200/test_full.txt',
    'datareader_class_name': 'SentenceBankDataReader',
    'dataset_class_name': 'MVPDataset',
    'input_max_length': 512,
    'output_max_length': 128,
    'nlp': 'ru_core_news_lg'},
   'dataloader': {'batch_size': 16, 'num_worke

In [5]:
tokenizer = AutoTokenizer.from_pretrained(train_recipe["model"]["hub_tokenizer_checkpoint"])
test_dataloader = DataModule.get_dataloader(train_recipe["model"], inference_recipe["dataloader"])

{'dataset': {'data_path': '/home/khaymonenko/aste/data/bank_3200/test_full.txt', 'datareader_class_name': 'SentenceBankDataReader', 'dataset_class_name': 'MVPDataset', 'input_max_length': 512, 'output_max_length': 128, 'nlp': 'ru_core_news_lg'}, 'dataloader': {'batch_size': 16, 'num_workers': 8}}


In [6]:
device = torch.device("cuda:7")

model = AutoModelForSeq2SeqLM.from_pretrained(train_recipe["model"]["hub_model_checkpoint"]).to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
      

In [86]:
batch = next(iter(test_dataloader))

with torch.no_grad():
    generation = model.generate(batch["source_ids"].to(device), num_beams=3, output_scores=True, temperature=0.01, return_dict_in_generate=True)

RuntimeError: CUDA out of memory. Tried to allocate 960.00 MiB (GPU 7; 31.75 GiB total capacity; 20.06 GiB already allocated; 928.50 MiB free; 21.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [81]:
generation

GreedySearchEncoderDecoderOutput(sequences=tensor([[    0,  1059,    51,    57,    50,  4747,  1059,    51,    57,    50,
           309,  1939,  1059, 22580,  6130,  1059, 22580,  2791,  1059,    51],
        [    0, 23568, 42212,    50,  1624,   988,  1902, 11595,    38,   784,
          1902, 11595,    38,  1373,   575,  1902, 11595,    38,   575,  1902],
        [    0,    16,    52,    16,    37,    16,    52,    16,    37,    16,
            52,    16,    37,    16,    52,    16,    37,    16,    52,    16],
        [    0,    16,    37,    16,    52,    16,    52,    16,    52,    16,
            52,    16,    52,    16,    52,    16,    52,    16,    52,    16],
        [    0,  1059,    51,    57,    50,  1492,    32,    51,    34,    37,
            16,    52,    16,    51,     1,    37,    16,    52,    16,    51],
        [    0,   969, 19789,    56,  5809,  1059,    51,    57,    50,  1624,
           374,   715,    39, 18689,    46,   309,  5734, 40016,    46,  2083],
   

In [85]:
len(generation["scores"])

19